In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for i,filename in enumerate(filenames):
        if i % 5000 == 0:
            print(i,os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Paths from your Kaggle dataset
CSV_PATH   = "/kaggle/input/militaryaircraftdetectiondataset/labels_with_split.csv"
IMAGES_DIR = "/kaggle/input/militaryaircraftdetectiondataset/dataset"  # folder with all images

# Working dir where we'll assemble YOLO data
WORK_DIR   = "/kaggle/working"
YOLO_DIR   = f"{WORK_DIR}/yolo_data"

In [3]:

!pip -q install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.6 MB/s eta 0:00:00:00:0100:01


In [4]:
import os, shutil, glob
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm

# Create split directories
for split in ["train", "val", "test"]:
    (Path(YOLO_DIR)/split/"images").mkdir(parents=True, exist_ok=True)
    (Path(YOLO_DIR)/split/"labels").mkdir(parents=True, exist_ok=True)

# Read CSV
df = pd.read_csv(CSV_PATH)

# Normalize column names just in case
df.columns = [c.strip().lower() for c in df.columns]

# Some datasets use 'valid' -> map to 'val'
df["split"] = df["split"].str.lower().replace({"valid":"val", "validation":"val"})

# Build list of available images (map both stem and filename → full path) to be robust to missing extensions in CSV
valid_exts = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}
all_image_paths = []
for p in Path(IMAGES_DIR).rglob("*"):
    if p.suffix.lower() in valid_exts:
        all_image_paths.append(p)

by_stem     = {p.stem: p for p in all_image_paths}
by_basename = {p.name: p  for p in all_image_paths}

def resolve_image_path(name: str):
    name = str(name)
    # if full basename (with ext) is present
    if name in by_basename: 
        return by_basename[name]
    # if only stem is there
    stem = Path(name).stem
    if stem in by_stem:
        return by_stem[stem]
    return None

df["class"] = "plane"
# Sanitize class names and build class map (sorted for deterministic ids)
df["class"] = df["class"].astype(str).str.strip()
class_names = sorted(df["class"].unique().tolist())
name_to_id  = {n:i for i,n in enumerate(class_names)}
print("Classes:", name_to_id)

Classes: {'plane': 0}


In [5]:
# Clip boxes helper
def xyxy_to_yolo(xmin, ymin, xmax, ymax, W, H):
    # clip to image bounds
    xmin = np.clip(xmin, 0, W-1)
    ymin = np.clip(ymin, 0, H-1)
    xmax = np.clip(xmax, 0, W-1)
    ymax = np.clip(ymax, 0, H-1)
    bw = max(0.0, xmax - xmin)
    bh = max(0.0, ymax - ymin)
    if bw <= 0 or bh <= 0:
        return None
    x_center = (xmin + xmax) / 2.0 / W
    y_center = (ymin + ymax) / 2.0 / H
    bw /= W
    bh /= H
    return x_center, y_center, bw, bh

# Group rows per (split, filename)
bad, written = 0, 0
for (split, fname), g in tqdm(df.groupby(["split", "filename"]), total=df.groupby(["split", "filename"]).ngroups):
    img_path = resolve_image_path(fname)
    if img_path is None:
        bad += 1
        continue

    # Label file path
    stem = img_path.stem
    lbl_fp = Path(YOLO_DIR)/split/"labels"/f"{stem}.txt"

    # Build label lines
    lines = []
    W = float(g.iloc[0]["width"])
    H = float(g.iloc[0]["height"])
    for _, r in g.iterrows():
        cid = name_to_id[str(r["class"])]
        xywh = xyxy_to_yolo(float(r["xmin"]), float(r["ymin"]), float(r["xmax"]), float(r["ymax"]), W, H)
        if xywh is None:
            continue
        lines.append(f"{cid} " + " ".join(f"{v:.6f}" for v in xywh))

    if not lines:
        # Skip images that ended up without any valid boxes
        continue

    # Write label file
    with open(lbl_fp, "w") as f:
        f.write("\n".join(lines))
    written += 1

    # Copy/link image to split/images
    out_img = Path(YOLO_DIR)/split/"images"/img_path.name
    try:
        os.link(img_path, out_img)     # hard link (fast, saves space)
    except Exception:
        try:
            os.symlink(img_path, out_img)  # symlink fallback
        except Exception:
            shutil.copy2(img_path, out_img)  # last resort: copy

print(f"Images with labels written: {written}, missing/failed: {bad}")

100%|██████████| 21950/21950 [00:13<00:00, 1622.52it/s]

Images with labels written: 21950, missing/failed: 0


In [6]:
yaml_text = f"""# Auto-generated for YOLOv8
path: {Path(YOLO_DIR).resolve()}    # dataset root
train: train/images
val:   val/images
test:  test/images

nc: {len(class_names)}
names: {class_names}
"""

with open(Path(YOLO_DIR)/"data.yaml", "w") as f:
    f.write(yaml_text)

print(open(Path(YOLO_DIR)/"data.yaml").read())

# Auto-generated for YOLOv8
path: /kaggle/working/yolo_data    # dataset root
train: train/images
val:   val/images
test:  test/images

nc: 1
names: ['plane']



In [7]:
from ultralytics import YOLO

# Choose a model size: yolov8n.pt / yolov8s.pt / yolov8m.pt / yolov8l.pt / yolov8x.pt
model = YOLO("yolov8s.pt")   # pretrained COCO weights

results = model.train(
    data=str(Path(YOLO_DIR)/"data.yaml"),
    imgsz=640,
    epochs=5,
    batch=-1,          # auto batch
    device=0,
    workers=8,
    project="runs",
    name="yolov8n-military",
    cos_lr=True,
    patience=30,
    close_mosaic=10,   # stabilize late training
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.203 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/yolo_data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, io

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       4026       7064      0.922      0.836      0.911      0.734
Speed: 0.1ms preprocess, 3.0ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /kaggle/working/runs/yolov8n-military


In [8]:
dir_to_zip = "/kaggle/working/runs"
import shutil
shutil.make_archive("out", 'zip', dir_to_zip)

'/kaggle/working/out.zip'